In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid",{'axes.grid' : False})

# Environment settings: 
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

In [61]:
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

In [62]:
from scipy import stats
import pandas_profiling as pf

In [63]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_train.shape , df_test.shape

((100000, 28), (50000, 27))

In [64]:
(df_train.columns[:-1]!=df_test.columns).sum()

0

In [65]:
df = pd.concat([df_train, df_test], ignore_index=True)
df = df.applymap(lambda x: x if x is np.NaN or not isinstance(x, str) else str(x).strip('_ ,"')
                      ).replace(' and', '').replace(['', 'nan', '!@9#%8', '#F%$D@*&8'], np.NaN)
df

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,...,NaN,809.98,26.823,22 Years and 1 Months,No,49.575,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.945,NaN,No,49.575,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609,22 Years and 3 Months,No,49.575,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.378,22 Years and 4 Months,No,49.575,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,...,Good,809.98,24.797,22 Years and 5 Months,No,49.575,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0x25fe5,CUS_0x8600,December,Sarah McBridec,4975,031-35-0942,Architect,20002.88,1929.907,10,...,NaN,3571.7,34.781,NaN,Yes,60.965,146.48632477751087,Low_spent_Small_value_payments,275.53956951573343,NaN
149996,0x25fee,CUS_0x942c,September,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,Good,502.38,27.759,31 Years and 11 Months,NM,35.104,181.44299902757518,Low_spent_Small_value_payments,409.39456169535066,NaN
149997,0x25fef,CUS_0x942c,October,Nicks,25,078-73-5990,Mechanic,39628.99,3359.416,4,...,Good,502.38,36.859,32 Years and 0 Months,No,35.104,10000,Low_spent_Large_value_payments,349.7263321025098,NaN
149998,0x25ff0,CUS_0x942c,November,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,Good,502.38,39.140,32 Years and 1 Months,No,35.104,97.59857973344877,High_spent_Small_value_payments,463.23898098947717,NaN


**EDA**

In [66]:
df.select_dtypes(include='O').nunique()

ID                         150000
Customer_ID                 12500
Month                          12
Name                        10128
Age                          2444
SSN                         12500
Occupation                     15
Annual_Income               13988
Num_of_Loan                   600
Type_of_Loan                 6260
Num_of_Delayed_Payment       1014
Changed_Credit_Limit         4604
Credit_Mix                      3
Outstanding_Debt            12203
Credit_History_Age            408
Payment_of_Min_Amount           3
Amount_invested_monthly    136497
Payment_Behaviour               6
Monthly_Balance            148224
Credit_Score                    3
dtype: int64

In [67]:
df.groupby('Month')['Customer_ID'].count().to_frame('count_Customer_ID').plot(kind ="bar")

In [68]:
# Drop the Columns we not use in model
df.drop(columns=['ID', 'Month', 'Name', 'SSN'], inplace=True)

In [69]:
df.select_dtypes(include='O').isna().sum()

Customer_ID                    0
Age                            0
Occupation                 10500
Annual_Income                  0
Num_of_Loan                    0
Type_of_Loan               17112
Num_of_Delayed_Payment     10500
Changed_Credit_Limit        3150
Credit_Mix                 30000
Outstanding_Debt               0
Credit_History_Age         13500
Payment_of_Min_Amount          0
Amount_invested_monthly     6750
Payment_Behaviour          11400
Monthly_Balance             1762
Credit_Score               50000
dtype: int64

**Assign Object Type to Numeric Types**

In [70]:
df.describe(include='O').T

,count,unique,top,freq
Customer_ID,150000,12500,CUS_0xd40,12
Age,150000,2444,39,4416
Occupation,139500,15,Lawyer,9899
Annual_Income,150000,13988,109945.32,24
Num_of_Loan,150000,600,3,22618
Type_of_Loan,132888,6260,Not Specified,2112
Num_of_Delayed_Payment,139500,1014,19,8188
Changed_Credit_Limit,146850,4604,11.5,197
Credit_Mix,120000,3,Standard,54858
Outstanding_Debt,150000,12203,1109.03,36


In [71]:
df['Customer_ID'] = df.Customer_ID.apply(lambda x: int(x[4:], 16))
df['Age'] = df.Age.astype(int)        
df['Annual_Income'] = df.Annual_Income.astype(float)
df['Num_of_Loan'] = df.Num_of_Loan.astype(int) 
df['Num_of_Delayed_Payment'] = df.Num_of_Delayed_Payment.astype(float)
df['Changed_Credit_Limit'] = df.Changed_Credit_Limit.astype(float)
df['Outstanding_Debt'] = df.Outstanding_Debt.astype(float)
df['Amount_invested_monthly'] = df.Amount_invested_monthly.astype(float)
df['Monthly_Balance'] = df.Monthly_Balance.astype(float)

Combining Object Columns

In [72]:
def Month_Converter(x):
    if pd.notnull(x):
        num1 = int(x.split(' ')[0])
        num2 = int(x.split(' ')[3])
      
        return (num1*12)+num2
    else:
        return x

In [73]:
df['Credit_History_Age'] = df.Credit_History_Age.apply(lambda x: Month_Converter(x)).astype(float)

In [74]:
df["Credit_History_Age"]

0        265.000
1            NaN
2        267.000
3        268.000
4        269.000
           ...  
149995       NaN
149996   383.000
149997   384.000
149998   385.000
149999   386.000
Name: Credit_History_Age, Length: 150000, dtype: float64

In [75]:
def get_Diff_Values_Colum(df_column, diff_value = []):   
    column = df_column.dropna()
    for i in column:
        if ',' not in i and i not in diff_value:
            diff_value.append(i)
        else:
            for data in map(lambda x:x.strip(), i.replace('and','').split(',')):
                if not data in diff_value:
                    diff_value.append(data)
    return dict(enumerate(sorted(diff_value)))

In [76]:
get_Diff_Values_Colum(df['Type_of_Loan'])

{0: 'Auto Loan',
 1: 'Credit-Builder Loan',
 2: 'Debt Consolidation Loan',
 3: 'Home Equity Loan',
 4: 'Mortgage Loan',
 5: 'Not Specified',
 6: 'Payday Loan',
 7: 'Personal Loan',
 8: 'Student Loan'}

**Detect Group NAN Values and Fill Group Mode**

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               150000 non-null  int64  
 1   Age                       150000 non-null  int64  
 2   Occupation                139500 non-null  object 
 3   Annual_Income             150000 non-null  float64
 4   Monthly_Inhand_Salary     127500 non-null  float64
 5   Num_Bank_Accounts         150000 non-null  int64  
 6   Num_Credit_Card           150000 non-null  int64  
 7   Interest_Rate             150000 non-null  int64  
 8   Num_of_Loan               150000 non-null  int64  
 9   Type_of_Loan              132888 non-null  object 
 10  Delay_from_due_date       150000 non-null  int64  
 11  Num_of_Delayed_Payment    139500 non-null  float64
 12  Changed_Credit_Limit      146850 non-null  float64
 13  Num_Credit_Inquiries      147000 non-null  f

Object Columns : Get Group Mode and Reassign NaN values

In [78]:
# Reassign and Show Function
def Reassign_Group_Object_NaN_Values(df, groupby, column, inplace=True):      
    import scipy.stats as stats
    # Assigning Wrong values Make Simple Function
    def make_NaN_and_fill_mode(df, groupby, column, inplace=True):
        # Assign None to np.NaN
        if df[column].isin([None]).sum():
            df[column][df[column].isin([None])] = np.NaN
            
        # fill with local mode
        result = df.groupby(groupby)[column].transform(lambda x: x.fillna(stats.mode(x)[0][0]))

        # inplace
        if inplace:
            df[column]=result
        else:
            return result
    
    # Run and show some info     
    if inplace:  
        # Before Assigning Wrong values      
        x = df[column].value_counts(dropna=False).head(1)
        print(f'\nBefore Assigning NaN {column}:', [x if x.index[0] is np.NaN or x.index[0] is None else 'No NaN Value'] , end='\n')
        a = df.groupby(groupby)[column].apply(list) 
        print(f'\nBefore Assigning Example {column}:\n', *a.head().values, sep='\n', end='\n')
        
        # Assigning
        make_NaN_and_fill_mode(df, groupby, column, inplace)
        
        # After Assigning Wrong values
        y = df[column].value_counts(dropna=False).head(1)
        print(f'\nAfter Assigning NaN {column}:', [y if y.index[0] is np.NaN or x.index[0] is None else 'No NaN Value'] , end='\n')
        b = df.groupby(groupby)[column].apply(list)
        print(f'\nAfter Assigning Example {column}:\n', *b.head().values, sep='\n', end='\n')
    else:   
        # Show
        return make_NaN_and_fill_mode(df, groupby, column, inplace)


In [79]:
df.describe(include='O').T

,count,unique,top,freq
Occupation,139500,15,Lawyer,9899
Type_of_Loan,132888,6260,Not Specified,2112
Credit_Mix,120000,3,Standard,54858
Payment_of_Min_Amount,150000,3,Yes,78484
Payment_Behaviour,138600,6,Low_spent_Small_value_payments,38207
Credit_Score,100000,3,Standard,53174


In [80]:
Reassign_Group_Object_NaN_Values(df, 'Customer_ID', 'Occupation')
Reassign_Group_Object_NaN_Values(df, 'Customer_ID', 'Credit_Mix')
Reassign_Group_Object_NaN_Values(df, 'Customer_ID', 'Payment_Behaviour')


Before Assigning NaN Occupation: [NaN    10500
Name: Occupation, dtype: int64]

Before Assigning Example Occupation:

['Journalist', 'Journalist', 'Journalist', 'Journalist', 'Journalist', nan, 'Journalist', 'Journalist', 'Journalist', 'Journalist', 'Journalist', 'Journalist']
['Manager', 'Manager', nan, 'Manager', 'Manager', 'Manager', 'Manager', 'Manager', 'Manager', 'Manager', 'Manager', 'Manager']
['Developer', 'Developer', 'Developer', 'Developer', 'Developer', 'Developer', 'Developer', 'Developer', 'Developer', 'Developer', 'Developer', 'Developer']
['Accountant', nan, 'Accountant', 'Accountant', 'Accountant', 'Accountant', 'Accountant', 'Accountant', 'Accountant', nan, 'Accountant', 'Accountant']
['Writer', 'Writer', 'Writer', 'Writer', nan, 'Writer', 'Writer', 'Writer', nan, 'Writer', 'Writer', 'Writer']

After Assigning NaN Occupation: ['No NaN Value']

After Assigning Example Occupation:

['Journalist', 'Journalist', 'Journalist', 'Journalist', 'Journalist', 'Journalist', 'J

In [81]:
df[df['Customer_ID']==1008]['Type_of_Loan']

34144     NaN
34145     NaN
34146     NaN
34147     NaN
34148     NaN
34149     NaN
34150     NaN
34151     NaN
117072    NaN
117073    NaN
117074    NaN
117075    NaN
Name: Type_of_Loan, dtype: object

In [82]:
df['Type_of_Loan'] = df['Type_of_Loan'].replace([', and ', ', '], ',', regex=True)

Numeric Columns : Get Group Min-Max Detect Outlier - Wrong Values and Reassign NaN values

In [83]:
# Reassign Wrong Values and Show Function
def Reassign_Group_Numeric_Wrong_Values(df, groupby, column, inplace=True):      
    import scipy.stats as stats 

    # Identify Wrong values Range
    def get_group_min_max(df, groupby, column):            
        cur = df[df[column].notna()].groupby(groupby)[column].apply(list)
        x, y = cur.apply(lambda x: stats.mode(x)).apply([min, max])
        return x[0][0], y[0][0]
    
    # Assigning Wrong values
    def make_group_NaN_and_fill_mode(df, groupby, column, inplace=True):
        df_dropped = df[df[column].notna()].groupby(groupby)[column].apply(list)
        x, y = df_dropped.apply(lambda x: stats.mode(x)).apply([min, max])
        mini, maxi = x[0][0], y[0][0]

        # assign Wrong Values to NaN
        col = df[column].apply(lambda x: np.NaN if ((x<mini)|(x>maxi)) else x)

        # fill with local mode
        mode_by_group = df.groupby(groupby)[column].transform(lambda x: x.mode()[0] if not x.mode().empty else np.NaN)
        result = col.fillna(mode_by_group)

        # inplace
        if inplace:
            df[column]=result
        else:
            return result
        
    
    # Run      
    if inplace:   
        print("\nExisting Min, Max Values:", df[column].apply([min, max]), sep='\n', end='\n')       
        mini, maxi = get_group_min_max(df, groupby, column)        
        print(f"\nGroupby by {groupby}'s Actual min, max Values:", f'min:\t{mini},\nmax:\t{ maxi}', sep='\n', end='\n')
        
        # Before Assigning Wrong values      
        x = df[column].value_counts(dropna=False).head(1)
        print(f'\nBefore Assigning Min, Max {column}:', [x if x.index[0] is np.NaN or x.index[0] is None else 'No NaN Value'] , end='\n')
        
        a = df.groupby(groupby)[column].apply(list) 
        print(f'\nBefore Assigning Example {column}:\n', *a.head().values, sep='\n', end='\n')
        
        # Assigning
        make_group_NaN_and_fill_mode(df, groupby, column, inplace)
        
        # After Assigning Wrong values
        y = df[column].value_counts(dropna=False).head(1)
        print(f'\nAfter Assigning  Min, Max {column}:', [y if y.index[0] is np.NaN or x.index[0] is None else 'No NaN Value'] , end='\n')
        
        b = df.groupby(groupby)[column].apply(list)
        print(f'\nAfter Assigning Example {column}:\n', *b.head().values, sep='\n', end='\n')
    else:   
        # Show
        return make_group_NaN_and_fill_mode(df, groupby, column, inplace)


In [84]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Customer_ID,150000.000,25982.667,14340.519,1006.000,13664.500,25777.000,38385.000,50999.000
Age,150000.000,110.338,684.067,-500.000,25.000,33.000,42.000,8698.000
Annual_Income,150000.000,173055.203,1404214.603,7005.930,19455.490,37578.610,72796.905,24198062.000
Monthly_Inhand_Salary,127500.000,4190.115,3180.490,303.645,1625.266,3091.000,5948.455,15204.633
Num_Bank_Accounts,150000.000,17.007,117.069,-1.000,3.000,6.000,7.000,1798.000
Num_Credit_Card,150000.000,22.623,129.143,0.000,4.000,5.000,7.000,1499.000
Interest_Rate,150000.000,71.235,461.537,1.000,8.000,13.000,20.000,5799.000
Num_of_Loan,150000.000,3.141,63.911,-100.000,1.000,3.000,5.000,1496.000
Delay_from_due_date,150000.000,21.063,14.860,-5.000,10.000,18.000,28.000,67.000
Num_of_Delayed_Payment,139500.000,30.912,224.534,-3.000,9.000,14.000,18.000,4399.000


In [85]:
df.select_dtypes(include='number').isna().sum()

Customer_ID                     0
Age                             0
Annual_Income                   0
Monthly_Inhand_Salary       22500
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Delay_from_due_date             0
Num_of_Delayed_Payment      10500
Changed_Credit_Limit         3150
Num_Credit_Inquiries         3000
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age          13500
Total_EMI_per_month             0
Amount_invested_monthly      6750
Monthly_Balance              1762
dtype: int64

In [86]:
# reassign Group nan and Wrong-outlier values
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Age')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Annual_Income')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Monthly_Inhand_Salary')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Bank_Accounts')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Credit_Card')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Interest_Rate')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_of_Loan')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Delay_from_due_date')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_of_Delayed_Payment')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Changed_Credit_Limit')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Credit_Inquiries')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Outstanding_Debt')
df['Credit_Utilization_Ratio'] = df.groupby('Customer_ID')['Credit_Utilization_Ratio'].apply(lambda x: x.interpolate().bfill().ffill())
df['Credit_History_Age'] = df.groupby('Customer_ID')['Credit_History_Age'].apply(lambda x: x.interpolate().bfill().ffill())
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Total_EMI_per_month')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Amount_invested_monthly')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Monthly_Balance')


Existing Min, Max Values:
min    -500
max    8698
Name: Age, dtype: int64

Groupby by Customer_ID's Actual min, max Values:
min:	14,
max:	56

Before Assigning Min, Max Age: ['No NaN Value']

Before Assigning Example Age:

[37, 38, 38, 8153, 38, 38, 38, 38, 38, 38, 38, 38]
[48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 49]
[3452, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37]
[22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23]
[43, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44]

After Assigning  Min, Max Age: ['No NaN Value']

After Assigning Example Age:

[37.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0]
[48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 49.0]
[37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0]
[22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 23.0, 23.0, 23.0, 23.0, 23.0]
[43.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0]

Existing Min, Max Values:
min       7005.930
max   24198062.000
Name: Annual_Incom

In [87]:
df['Monthly_Balance'][df['Monthly_Balance']<0]

5545     -333333333333333314856026112.000
26177    -333333333333333314856026112.000
29158    -333333333333333314856026112.000
35570    -333333333333333314856026112.000
38622    -333333333333333314856026112.000
41828    -333333333333333314856026112.000
60009    -333333333333333314856026112.000
75251    -333333333333333314856026112.000
82918    -333333333333333314856026112.000
83255    -333333333333333314856026112.000
111363   -333333333333333314856026112.000
117036   -333333333333333314856026112.000
120915   -333333333333333314856026112.000
136273   -333333333333333314856026112.000
141458   -333333333333333314856026112.000
144055   -333333333333333314856026112.000
149501   -333333333333333314856026112.000
Name: Monthly_Balance, dtype: float64

In [88]:
df.loc[df['Monthly_Balance']<0, 'Monthly_Balance'] = None

In [89]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Monthly_Balance')


Existing Min, Max Values:
min      0.008
max   1183.931
Name: Monthly_Balance, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	0.007759664775335295,
max:	1183.9306960885192

Before Assigning Min, Max Monthly_Balance: ['No NaN Value']

Before Assigning Example Monthly_Balance:

[310.39167595729333, 295.61432104073225, 333.9600295968694, 309.1977625633686, 323.9664999631732, 334.864456417663, 280.04409663983637, 328.97449591530835, 279.14863869481445, 306.61186979020056, 301.87706575887506, 227.25690402118008]
[323.5550777151861, 309.6085947906515, 265.4874646531188, 291.11606352715245, 245.61898478406326, 239.46481499922072, 334.61958840607645, 275.3861034815317, 279.58867604354555, 318.4767883518287, 260.7342254927612, 307.57591763672826]
[406.62288544232047, 350.39351957834924, 218.30384271186168, 354.63462349002145, 441.30551404075095, 420.04620414440666, 337.6532108572321, 370.2991478574313, 474.5734332336703, 439.0620993738886, 203.58056486748768, 459.8749140

In [90]:
df.isna().sum()

Customer_ID                     0
Age                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary           0
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                17112
Delay_from_due_date             0
Num_of_Delayed_Payment          0
Changed_Credit_Limit            0
Num_Credit_Inquiries            0
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age              0
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly         0
Payment_Behaviour               0
Monthly_Balance                 0
Credit_Score                50000
dtype: int64

In [91]:

# Drop the Columns we not use in model
df.drop(columns=['Customer_ID'], inplace=True)

In [92]:
# Drop the Rows we can not use in model
df = df[df['Credit_Score'].notna()]
df.shape

(100000, 23)

In [93]:
# Correlation between Numerical features
plt.figure(figsize=(14,8))
sns.heatmap(
    df.corr(), 
    mask=np.triu(np.ones_like(df.corr(), dtype=bool)), 
    annot=True, vmin=-1, vmax=1, cmap="PiYG"
)

Classification 

In [94]:
# import libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline

# Superviseed Classifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from xgboost import XGBClassifier

# Supervised Classifier- metrix
from  sklearn.metrics import accuracy_score, recall_score, make_scorer, precision_score, f1_score,log_loss
from sklearn.metrics import classification_report, plot_confusion_matrix,confusion_matrix
from sklearn.metrics import plot_roc_curve, roc_curve, roc_auc_score, auc
from sklearn.metrics import plot_precision_recall_curve, precision_recall_curve,average_precision_score

In [95]:
# supervised-Cross_validation-GridSearchCV
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [96]:
import pickle
random_state = 42

In [100]:
df1 = df.copy()

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Age                       100000 non-null  float64
 1   Occupation                100000 non-null  object 
 2   Annual_Income             100000 non-null  float64
 3   Monthly_Inhand_Salary     100000 non-null  float64
 4   Num_Bank_Accounts         100000 non-null  float64
 5   Num_Credit_Card           100000 non-null  float64
 6   Interest_Rate             100000 non-null  float64
 7   Num_of_Loan               100000 non-null  float64
 8   Type_of_Loan              88592 non-null   object 
 9   Delay_from_due_date       100000 non-null  float64
 10  Num_of_Delayed_Payment    100000 non-null  float64
 11  Changed_Credit_Limit      100000 non-null  float64
 12  Num_Credit_Inquiries      100000 non-null  float64
 13  Credit_Mix                100000 non-null  ob

Train | Test split

In [102]:
X = df.drop(columns="Credit_Score")
y = df['Credit_Score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

X_train.shape, X_test.shape, y_train.shape, 

((80000, 22), (20000, 22), (80000,))

Dummy Operation

In [103]:
X_train = X_train.join(X_train["Type_of_Loan"].str.get_dummies(sep = ",").add_prefix("tol_"))
X_test = X_test.join(X_test["Type_of_Loan"].str.get_dummies(sep = ",").add_prefix("tol_"))
X_test = X_test.reindex(columns = X_train.columns, fill_value=0) # "0"

X_train.drop(columns=["Type_of_Loan"], inplace = True)
X_test.drop(columns=["Type_of_Loan"], inplace = True)

X_train.shape, X_test.shape

((80000, 30), (20000, 30))

OneHotEncoder and LabelEncoder

In [104]:
from sklearn.preprocessing import OneHotEncoder

cat = X_train.select_dtypes(exclude="number").columns.to_list()    
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)

X_train_cat = pd.DataFrame(
    enc.fit_transform(X_train[cat]), index = X_train.index, 
    columns = enc.get_feature_names_out(cat)
)    
X_test_cat  = pd.DataFrame(
    enc.transform(X_test[cat]), index = X_test.index, 
    columns = enc.get_feature_names_out(cat)
)    
X_train_enc = X_train_cat.join(X_train.select_dtypes("number"))
X_test_enc = X_test_cat.join(X_test.select_dtypes("number"))

X_train_enc.shape, X_test_enc.shape

((80000, 53), (20000, 53))

In [105]:
X_train_enc.columns

Index(['Occupation_Accountant', 'Occupation_Architect', 'Occupation_Developer',
       'Occupation_Doctor', 'Occupation_Engineer', 'Occupation_Entrepreneur',
       'Occupation_Journalist', 'Occupation_Lawyer', 'Occupation_Manager',
       'Occupation_Mechanic', 'Occupation_Media_Manager',
       'Occupation_Musician', 'Occupation_Scientist', 'Occupation_Teacher',
       'Occupation_Writer', 'Credit_Mix_Bad', 'Credit_Mix_Good',
       'Credit_Mix_Standard', 'Payment_of_Min_Amount_NM',
       'Payment_of_Min_Amount_No', 'Payment_of_Min_Amount_Yes',
       'Payment_Behaviour_High_spent_Large_value_payments',
       'Payment_Behaviour_High_spent_Medium_value_payments',
       'Payment_Behaviour_High_spent_Small_value_payments',
       'Payment_Behaviour_Low_spent_Large_value_payments',
       'Payment_Behaviour_Low_spent_Medium_value_payments',
       'Payment_Behaviour_Low_spent_Small_value_payments', 'Age',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Nu

In [106]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test  = le.transform(y_test)

pd.DataFrame(y_train).value_counts().sort_index()

0    14301
1    23124
2    42575
dtype: int64

Scale data

In [107]:
scaler = MinMaxScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_enc), columns=X_train_enc.columns)
X_test_scaled  = pd.DataFrame(scaler.transform(X_test_enc), columns=X_test_enc.columns)

Now Data is ready to apply algorithm (classification)

1) Implement Logistic Regression Model

In [108]:
log_model = LogisticRegression(
    class_weight = "balanced",
    random_state=random_state
)

In [109]:
log_model.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', random_state=42)

In [110]:
y_pred_train = log_model.predict(X_train_scaled)
y_pred_test  = log_model.predict(X_test_scaled)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.50      0.84      0.62     14301
           1       0.64      0.68      0.66     23124
           2       0.82      0.60      0.69     42575

    accuracy                           0.67     80000
   macro avg       0.65      0.71      0.66     80000
weighted avg       0.71      0.67      0.67     80000

              precision    recall  f1-score   support

           0       0.50      0.83      0.62      3527
           1       0.64      0.69      0.66      5874
           2       0.81      0.60      0.69     10599

    accuracy                           0.67     20000
   macro avg       0.65      0.71      0.66     20000
weighted avg       0.71      0.67      0.67     20000



Best Parameters (GridSearchCV)

In [111]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

model = LogisticRegression(    
    class_weight = "balanced",
    max_iter=10000,
    random_state=random_state
)
param_grid = {
    "penalty" : ['l1'],
    "C" : [0.1],
    "solver" : ['saga'], # 'liblinear', 
}
# Identify people with low credit scores
recall_poor = make_scorer(recall_score, average = None, labels=[1])
grid_model_log = GridSearchCV(
    estimator=model,
    param_grid = param_grid, 
    scoring=recall_poor, 
    error_score="raise",
    n_jobs=-1,
    cv=5
) 


In [112]:
%%time
grid_model_log.fit(X_train_scaled, y_train)

CPU times: user 4.33 s, sys: 110 ms, total: 4.44 s
Wall time: 20.5 s


GridSearchCV(cv=5, error_score='raise',
             estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=10000, random_state=42),
             n_jobs=-1,
             param_grid={'C': [0.1], 'penalty': ['l1'], 'solver': ['saga']},
             scoring=make_scorer(recall_score, average=None, labels=[1]))

In [113]:
grid_model_log.best_params_

{'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}

In [114]:
grid_model_log.best_score_

0.6803752735434396

In [115]:
y_pred_train = grid_model_log.predict(X_train_scaled)
y_pred_test  = grid_model_log.predict(X_test_scaled)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.49      0.84      0.62     14301
           1       0.64      0.68      0.66     23124
           2       0.82      0.60      0.69     42575

    accuracy                           0.67     80000
   macro avg       0.65      0.71      0.66     80000
weighted avg       0.71      0.67      0.67     80000

              precision    recall  f1-score   support

           0       0.50      0.83      0.62      3527
           1       0.64      0.69      0.66      5874
           2       0.81      0.60      0.69     10599

    accuracy                           0.67     20000
   macro avg       0.65      0.71      0.66     20000
weighted avg       0.71      0.67      0.67     20000



2) Random Forest Classifier Model

*OrdinalEncoder*

In [116]:
from sklearn.preprocessing import OrdinalEncoder

cat = X_train.select_dtypes(exclude="number").columns.to_list()    
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train[cat] = enc.fit_transform(X_train[cat])
X_test[cat] = enc.transform(X_test[cat])

X_train.shape, X_test.shape

((80000, 30), (20000, 30))

*RandomForestClassifier*

In [117]:
from sklearn.ensemble import RandomForestClassifier

rfc_model = RandomForestClassifier(
    class_weight = 'balanced',
    random_state=random_state
)

In [118]:
%%time
rfc_model.fit(X_train, y_train)

CPU times: user 33.2 s, sys: 205 ms, total: 33.4 s
Wall time: 35.1 s


RandomForestClassifier(class_weight='balanced', random_state=42)

In [119]:
y_pred_train = rfc_model.predict(X_train)
y_pred_test  = rfc_model.predict(X_test)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14301
           1       1.00      1.00      1.00     23124
           2       1.00      1.00      1.00     42575

    accuracy                           1.00     80000
   macro avg       1.00      1.00      1.00     80000
weighted avg       1.00      1.00      1.00     80000

              precision    recall  f1-score   support

           0       0.79      0.76      0.77      3527
           1       0.80      0.84      0.82      5874
           2       0.83      0.82      0.83     10599

    accuracy                           0.82     20000
   macro avg       0.81      0.81      0.81     20000
weighted avg       0.82      0.82      0.82     20000



GridsearchCV

In [120]:
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier(
    class_weight = 'balanced',
    random_state=random_state
)
param_grid = {
    'max_depth': [3], 
    'n_estimators': [10],
    'max_features': [None],
}
# Identify people with low credit scores
recall_poor = make_scorer(recall_score, average = None, labels=[1])
grid_model_rfc = GridSearchCV(
    estimator=model,
    param_grid = param_grid, 
    error_score="raise",
    scoring=recall_poor,
    refit=True,
#     verbose=2,
    n_jobs=-1,
    cv=5,
)

In [121]:
%%time
grid_model_rfc.fit(X_train, y_train)

CPU times: user 2.98 s, sys: 39 ms, total: 3.02 s
Wall time: 13.7 s


GridSearchCV(cv=5, error_score='raise',
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [3], 'max_features': [None],
                         'n_estimators': [10]},
             scoring=make_scorer(recall_score, average=None, labels=[1]))

In [122]:
grid_model_rfc.best_params_

{'max_depth': 3, 'max_features': None, 'n_estimators': 10}

In [123]:
grid_model_rfc.best_score_

0.8050509959786776

In [124]:
y_pred_train = grid_model_rfc.predict(X_train)
y_pred_test  = grid_model_rfc.predict(X_test)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.52      0.75      0.62     14301
           1       0.61      0.80      0.69     23124
           2       0.85      0.57      0.69     42575

    accuracy                           0.67     80000
   macro avg       0.66      0.71      0.67     80000
weighted avg       0.72      0.67      0.68     80000

              precision    recall  f1-score   support

           0       0.52      0.75      0.61      3527
           1       0.61      0.80      0.69      5874
           2       0.85      0.57      0.68     10599

    accuracy                           0.67     20000
   macro avg       0.66      0.71      0.66     20000
weighted avg       0.72      0.67      0.67     20000



XGBoost Classifier

In [125]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    random_state=random_state,
)


In [126]:
from sklearn.utils import class_weight
classes_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)
classes_weights

array([0.62634566, 1.86467147, 1.86467147, ..., 0.62634566, 0.62634566,
       1.15320302])

In [127]:

comp = pd.DataFrame(classes_weights)

comp["label"] = pd.DataFrame(y_train).reset_index(drop=True)
comp.groupby("label")[0].value_counts()

label  0    
0      1.865    14301
1      1.153    23124
2      0.626    42575
Name: 0, dtype: int64

In [128]:
%%time
xgb_model.fit(
    X_train, y_train, 
    sample_weight=classes_weights
)
# weight parameter in XGBoost is per instance not per class.

CPU times: user 31.3 s, sys: 91.3 ms, total: 31.4 s
Wall time: 36.5 s


XGBClassifier(objective='multi:softprob', random_state=42)

In [129]:
y_pred_train = xgb_model.predict(X_train)
y_pred_test  = xgb_model.predict(X_test)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.51      0.85      0.64     14301
           1       0.65      0.78      0.71     23124
           2       0.86      0.58      0.69     42575

    accuracy                           0.68     80000
   macro avg       0.67      0.73      0.68     80000
weighted avg       0.74      0.68      0.69     80000

              precision    recall  f1-score   support

           0       0.51      0.84      0.64      3527
           1       0.64      0.77      0.70      5874
           2       0.85      0.57      0.69     10599

    accuracy                           0.68     20000
   macro avg       0.67      0.73      0.68     20000
weighted avg       0.73      0.68      0.68     20000



With Best Parameters (GridsearchCV)

In [130]:
from sklearn.model_selection import GridSearchCV

model = XGBClassifier(
    random_state=random_state
)
param_grid = {
    'n_estimators':[50, 100],
}
# Identify people with low credit scores
recall_poor = make_scorer(recall_score, average = None, labels=[1])
grid_model_xgb = GridSearchCV(
    estimator=model,
    param_grid = param_grid, 
    error_score="raise",
    scoring=recall_poor,
    refit=True,
    n_jobs=-1
)

In [131]:
%%time
grid_model_xgb.fit(
    X_train, y_train, 
    sample_weight=classes_weights
)

CPU times: user 25.9 s, sys: 188 ms, total: 26.1 s
Wall time: 2min 46s


GridSearchCV(error_score='raise', estimator=XGBClassifier(random_state=42),
             n_jobs=-1, param_grid={'n_estimators': [50, 100]},
             scoring=make_scorer(recall_score, average=None, labels=[1]))

In [132]:
grid_model_xgb.best_params_

{'n_estimators': 100}

In [133]:
grid_model_xgb.best_score_


0.7736982231366314

In [134]:
y_pred_train = grid_model_xgb.predict(X_train)
y_pred_test  = grid_model_xgb.predict(X_test)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.51      0.85      0.64     14301
           1       0.65      0.78      0.71     23124
           2       0.86      0.58      0.69     42575

    accuracy                           0.68     80000
   macro avg       0.67      0.73      0.68     80000
weighted avg       0.74      0.68      0.69     80000

              precision    recall  f1-score   support

           0       0.51      0.84      0.64      3527
           1       0.64      0.77      0.70      5874
           2       0.85      0.57      0.69     10599

    accuracy                           0.68     20000
   macro avg       0.67      0.73      0.68     20000
weighted avg       0.73      0.68      0.68     20000



**Future Importance XGBoost**
* Permutation Based Feature importance(with Scikit-learn)

In [135]:
# xgb_model has best scorer default parameters
xgb_model.feature_importances_

array([0.003919  , 0.00322602, 0.00684318, 0.00843683, 0.03316756,
       0.0387332 , 0.256837  , 0.01020552, 0.03400452, 0.01831861,
       0.02862972, 0.02900105, 0.23457249, 0.19550367, 0.00102424,
       0.01087553, 0.01708607, 0.01842079, 0.00925704, 0.01271737,
       0.00597547, 0.        , 0.00443327, 0.        , 0.00337085,
       0.00370014, 0.00159707, 0.00280818, 0.00733562, 0.        ],
      dtype=float32)

In [136]:
plt.figure(figsize=(14,8))
plt.barh(X_test.columns, xgb_model.feature_importances_)

<BarContainer object of 30 artists>

In [137]:
from sklearn.inspection import permutation_importance

perm_importance = permutation_importance(xgb_model, X_test, y_test)
perm_importance

{'importances_mean': array([ 6.0000e-05,  2.3000e-04,  7.6000e-04,  1.1900e-03,  4.7300e-03,
         1.2400e-03,  9.0820e-02,  4.0000e-04,  2.4110e-02,  4.1200e-03,
         1.3700e-03,  1.3300e-03,  1.6342e-01,  4.5150e-02, -2.0000e-05,
        -5.1000e-04, -3.0000e-05,  8.6200e-03, -1.5000e-04, -5.2000e-04,
         1.8000e-04,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.9000e-04,
         0.0000e+00,  4.0000e-05,  0.0000e+00,  2.7000e-04,  0.0000e+00]),
 'importances_std': array([1.24096736e-04, 7.48331477e-05, 2.13072758e-04, 1.82756669e-04,
        3.54400903e-04, 4.32897216e-04, 9.72419662e-04, 2.23606798e-04,
        5.32353266e-04, 4.86415460e-04, 1.63095064e-04, 4.34281015e-04,
        1.94128823e-03, 6.98569968e-04, 4.00000000e-05, 3.65239647e-04,
        7.48331477e-05, 7.99124521e-04, 2.23606798e-04, 3.41467422e-04,
        9.79795897e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        6.63324958e-05, 0.00000000e+00, 5.83095189e-05, 0.00000000e+00,
        5.0990195

In [138]:
sorted_idx = perm_importance.importances_mean.argsort()
plt.figure(figsize=(14,8))
plt.barh(X_test.columns[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

Text(0.5, 0, 'Permutation Importance')

In [139]:
X_test.columns

Index(['Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary',
       'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'tol_Auto Loan', 'tol_Credit-Builder Loan',
       'tol_Debt Consolidation Loan', 'tol_Home Equity Loan',
       'tol_Mortgage Loan', 'tol_Not Specified', 'tol_Payday Loan',
       'tol_Personal Loan', 'tol_Student Loan'],
      dtype='object')

Final Model

In [141]:
# we selected 8 features for final model prediciton
df_final = df[[
    'Credit_Mix', 'Interest_Rate', 'Outstanding_Debt', 'Delay_from_due_date', 
    'Total_EMI_per_month', 'Changed_Credit_Limit', 'Monthly_Inhand_Salary', 'Annual_Income',
    'Credit_Score'
]]
df_final

,Credit_Mix,Interest_Rate,Outstanding_Debt,Delay_from_due_date,Total_EMI_per_month,Changed_Credit_Limit,Monthly_Inhand_Salary,Annual_Income,Credit_Score
0,Good,3.000,809.980,3.000,49.575,11.270,1824.843,19114.120,Good
1,Good,3.000,809.980,-1.000,49.575,11.270,1824.843,19114.120,Good
2,Good,3.000,809.980,3.000,49.575,11.270,1824.843,19114.120,Good
3,Good,3.000,809.980,5.000,49.575,6.270,1824.843,19114.120,Good
4,Good,3.000,809.980,6.000,49.575,11.270,1824.843,19114.120,Good
...,...,...,...,...,...,...,...,...,...
99995,Good,7.000,502.380,23.000,35.104,11.500,3359.416,39628.990,Poor
99996,Good,7.000,502.380,18.000,35.104,11.500,3359.416,39628.990,Poor
99997,Good,7.000,502.380,27.000,35.104,11.500,3359.416,39628.990,Poor
99998,Good,7.000,502.380,20.000,35.104,11.500,3359.416,39628.990,Standard


In [142]:
X = df_final.drop(columns='Credit_Score')
y = df_final['Credit_Score']

In [143]:
cat = X.select_dtypes(exclude="number").columns.to_list()    
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X[cat] = enc.fit_transform(X[cat])

X.shape

(100000, 8)

In [144]:
le = LabelEncoder()
y = le.fit_transform(y)

In [145]:
classes_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y)
classes_weights

array([1.86971805, 1.86971805, 1.86971805, ..., 1.14950456, 0.62687278,
       1.14950456])

In [146]:
final_model = XGBClassifier(
    random_state=random_state,
)

In [147]:
%%time
final_model.fit(
    X, y, 
    sample_weight=classes_weights
)

CPU times: user 13 s, sys: 27.1 ms, total: 13 s
Wall time: 13 s


XGBClassifier(objective='multi:softprob', random_state=42)

In [148]:
y_pred = final_model.predict(X)

print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.85      0.63     17828
           1       0.65      0.75      0.69     28998
           2       0.86      0.58      0.69     53174

    accuracy                           0.68    100000
   macro avg       0.67      0.73      0.67    100000
weighted avg       0.73      0.68      0.68    100000



In [149]:
human_entry = {
 'Credit_Mix': 'Standard',
 'Interest_Rate': 15,
 'Outstanding_Debt': 1526,
 'Delay_from_due_date': 12,
 'Total_EMI_per_month': 81,
 'Changed_Credit_Limit': 5,
 'Monthly_Inhand_Salary': 6200,
 'Annual_Income': 68000,
}
human_entry = pd.DataFrame([human_entry])
human_entry[cat] = enc.transform(human_entry[cat])
human_entry

,Credit_Mix,Interest_Rate,Outstanding_Debt,Delay_from_due_date,Total_EMI_per_month,Changed_Credit_Limit,Monthly_Inhand_Salary,Annual_Income
0,2.000,15,1526,12,81,5,6200,68000


In [152]:
predict = final_model.predict(human_entry)
predict, le.inverse_transform(predict)

(array([1]), array(['Poor'], dtype=object))

Prepare Model Deployment

In [153]:
import pickle
pickle.dump(final_model, open("final_model_credit_score_pred", 'wb'))